In [1]:

%python start_system.py --skip-db-setup --skip-db-init --skip-db-seed

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [2]:
%python start_system.py

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [1]:
def add_to_autostart():
    """Додає скрипт в автозавантаження"""
    print_header("Додавання в автозавантаження")

    try:
        if platform.system() == 'Windows':
            startup = os.path.join(os.environ['APPDATA'], r'Microsoft\Windows\Start Menu\Programs\Startup')
            shortcut_path = os.path.join(startup, "ІАС НГУ.bat")

            python_path = sys.executable
            script_path = os.path.abspath(__file__)

            with open(shortcut_path, 'w', encoding='utf-8') as f:
                f.write(f'@echo off\n')
                f.write(f'echo Запуск ІАС НГУ...\n')
                f.write(f'"{python_path}" "{script_path}" --no-shortcut\n')
                f.write(f'pause\n')

            print(f"Ярлик для автозавантаження створено: {shortcut_path}")

        elif platform.system() == 'Linux':
            autostart_dir = os.path.expanduser("~/.config/autostart")
            os.makedirs(autostart_dir, exist_ok=True)

            shortcut_path = os.path.join(autostart_dir, "ias_ngu.desktop")

            python_path = sys.executable
            script_path = os.path.abspath(__file__)

            with open(shortcut_path, 'w', encoding='utf-8') as f:
                f.write("[Desktop Entry]\n")
                f.write("Type=Application\n")
                f.write("Name=ІАС НГУ\n")
                f.write(f"Exec={python_path} {script_path} --no-shortcut\n")
                f.write("Terminal=true\n")
                f.write("Comment=Інформаційно-аналітична система НГУ\n")

            print(f"Файл автозавантаження створено: {shortcut_path}")

        elif platform.system() == 'Darwin':
            login_items_script = f"""
            osascript -e 'tell application "System Events" to make login item at end with properties {{path:"{os.path.abspath(__file__)}", hidden:false}}'
            """
            os.system(login_items_script)
            print("Скрипт додано до автозавантаження macOS")

        else:
            print("Автозавантаження для цієї ОС не підтримується автоматично.")

        return True

    except Exception as e:
        print(f"Помилка при додаванні в автозавантаження: {e}")
        return False


In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output

# Ініціалізація додатку Dash
app = dash.Dash(__name__)

# Стилі
colors = {
    'visualization': '#FFF9C4',  # світло-жовтий
    'threats': '#FFCCBC',        # світло-оранжевий
    'resources': '#FFCDD2',      # світло-червоний
    'forecasting': '#F8BBD0',    # світло-рожевий
    'background': '#F5F5F5',     # світло-сірий фон
    'text': '#333333'            # темно-сірий текст
}

# Генерація тестових даних
def generate_test_data():
    # Дані для візуалізації (області України)
    regions = [
        'Київська', 'Львівська', 'Харківська', 'Одеська', 'Дніпропетровська',
        'Донецька', 'Луганська', 'Запорізька', 'Херсонська', 'Миколаївська',
        'Вінницька', 'Полтавська', 'Сумська', 'Чернігівська', 'Черкаська',
        'Хмельницька', 'Рівненська', 'Волинська', 'Івано-Франківська', 'Тернопільська',
        'Закарпатська', 'Чернівецька', 'Житомирська', 'Кіровоградська'
    ]
    # Рівень безпеки (вищі значення - безпечніше)
    security_levels = np.random.randint(10, 100, size=len(regions))
    # Області під високим ризиком (прикордонні з РФ та близькі до фронту)
    high_risk_regions = ['Харківська', 'Донецька', 'Луганська', 'Запорізька', 'Херсонська', 'Сумська', 'Чернігівська']
    for region in high_risk_regions:
        if region in regions:
            idx = regions.index(region)
            security_levels[idx] = np.random.randint(10, 40)  # Нижчий рівень безпеки
            
    visualization_data = pd.DataFrame({'Область': regions, 'Рівень безпеки': security_levels})
    
    # Дані для аналізу загроз (специфічні для Російсько-Української війни)
    threat_types = [
        'Ракетні удари', 'Артилерійські обстріли', 'Безпілотники', 'Кібератаки',
        'Диверсійні групи', 'Інформаційні операції', 'Інфраструктурні атаки'
    ]
    risk_levels = np.random.randint(3, 10, size=len(threat_types))
    threat_categories = [
        'Військова загроза', 'Військова загроза', 'Військова загроза', 'Кібер загроза',
        'Фізична загроза', 'Інформаційна загроза', 'Критична інфраструктура'
    ]
    threats_data = pd.DataFrame({
        'Тип загрози': threat_types, 
        'Рівень ризику': risk_levels,
        'Категорія': threat_categories
    })
    
    # Дані для управління ресурсами
    resources = [
        'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
        'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
    ]
    usage = np.random.randint(50, 100, size=len(resources))
    resources_data = pd.DataFrame({'Ресурс': resources, 'Використання (%)': usage})
    
    # Дані для прогнозування (наприклад, прогноз потреб або ризиків)
    dates = pd.date_range(start='2025-01-01', periods=30)
    forecasted = np.cumsum(np.random.normal(0, 2, size=len(dates))) + 50
    actual = forecasted[:20] + np.random.normal(0, 3, size=20)
    forecast_data = pd.DataFrame({
        'Дата': dates,
        'Прогноз': forecasted,
        'Фактично': np.append(actual, [None] * 10)  # 10 останніх днів не мають фактичних даних
    })
    
    return visualization_data, threats_data, resources_data, forecast_data

viz_data, threats_data, resources_data, forecast_data = generate_test_data()

# Створення графіків
def create_ukraine_map():
    # Спрощена карта України (heatmap)
    # Для кращої візуалізації використовуємо scatter-карту з індивідуальними мітками областей
    fig = go.Figure()
    
    # Додаємо контур для представлення України
    ukraine_lons = [22, 40, 40, 22, 22]  # Координати longitude
    ukraine_lats = [44, 44, 52, 52, 44]  # Координати latitude
    
    fig.add_trace(go.Scatter(
        x=ukraine_lons,
        y=ukraine_lats,
        mode='lines',
        line=dict(width=2, color='blue'),
        name='Контур України',
        showlegend=False
    ))
    
    # Додаємо точки для кожної області
    # Приблизні координати для областей (спрощено)
    region_coords = {
        'Київська': (30.5, 50.4),
        'Львівська': (24.0, 49.8),
        'Харківська': (36.2, 49.9),
        'Одеська': (30.7, 46.5),
        'Дніпропетровська': (35.0, 48.5),
        'Донецька': (37.8, 48.0),
        'Луганська': (39.3, 48.9),
        'Запорізька': (35.4, 47.8),
        'Херсонська': (33.5, 46.6),
        'Миколаївська': (32.0, 47.0),
        'Вінницька': (28.5, 49.2),
        'Полтавська': (34.6, 49.6),
        'Сумська': (34.8, 51.0),
        'Чернігівська': (31.3, 51.5),
        'Черкаська': (32.0, 49.4),
        'Хмельницька': (27.0, 49.4),
        'Рівненська': (26.2, 50.6),
        'Волинська': (25.0, 51.0),
        'Івано-Франківська': (24.7, 48.9),
        'Тернопільська': (25.6, 49.6),
        'Закарпатська': (22.3, 48.6),
        'Чернівецька': (25.9, 48.3),
        'Житомирська': (28.7, 50.3),
        'Кіровоградська': (32.3, 48.5)
    }
    
    # Додаємо маркери для областей з кольором залежно від рівня безпеки
    for i, row in viz_data.iterrows():
        region = row['Область']
        security = row['Рівень безпеки']
        
        if region in region_coords:
            lon, lat = region_coords[region]
            
            # Визначаємо колір (червоний для небезпечних, жовтий для середніх, зелений для безпечних)
            if security < 40:
                marker_color = 'red'
            elif security < 70:
                marker_color = 'orange'
            else:
                marker_color = 'green'
                
            fig.add_trace(go.Scatter(
                x=[lon],
                y=[lat],
                mode='markers+text',
                marker=dict(size=15, color=marker_color),
                text=region,
                textposition="top center",
                name=f"{region} ({security})",
                showlegend=False
            ))
    
    # Налаштування вигляду карти
    fig.update_layout(
        title='Оперативна ситуація в Україні',
        geo=dict(
            scope='europe',
            projection_type='mercator',
            showland=True,
            landcolor='rgb(240, 240, 240)',
            countrycolor='rgb(204, 204, 204)'
        ),
        margin=dict(l=0, r=0, t=30, b=0),
        # Встановлюємо обмежену область для відображення України
        xaxis=dict(range=[22, 40]),
        yaxis=dict(range=[44, 52], scaleanchor="x", scaleratio=1)
    )
    
    return fig

def create_threat_analysis():
    # Створюємо аналіз загроз з урахуванням Російсько-Української війни
    fig = px.bar(
        threats_data,
        x='Тип загрози',
        y='Рівень ризику',
        color='Категорія',
        color_discrete_map={
            'Військова загроза': '#FF5733',
            'Кібер загроза': '#33A1FF',
            'Фізична загроза': '#FF33FC',
            'Інформаційна загроза': '#33FF57',
            'Критична інфраструктура': '#FFD433'
        },
        title='Аналіз загроз (Російсько-Українська війна)'
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis=dict(tickangle=45),
        yaxis=dict(title='Рівень загрози (1-10)')
    )
    
    return fig

def create_resource_management():
    fig = go.Figure()
    for i, row in resources_data.iterrows():
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=row['Використання (%)'],
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': row['Ресурс']},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkred" if row['Використання (%)'] > 80 else "orange" if row['Використання (%)'] > 60 else "green"},
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ))
    fig.update_layout(
        grid={'rows': 3, 'columns': 3, 'pattern': "independent"},
        title='Використання ресурсів',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig

def create_forecast_chart():
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Фактично'],
        mode='markers+lines',
        name='Фактичні дані',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Прогноз'],
        mode='lines',
        name='Прогноз',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title='Прогнозування потреб у ресурсах',
        xaxis_title='Дата',
        yaxis_title='Значення',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1(
        'Компоненти панелі управління',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'fontFamily': 'Arial',
            'marginBottom': '30px'
        }
    ),
    
    # Ряд з чотирма компонентами
    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-around'}, children=[
        # Візуалізація
        html.Div(
            style={
                'backgroundColor': colors['visualization'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Візуалізація', style={'textAlign': 'center', 'color': '#B58500'}),
                html.P('Відображає оперативну ситуацію в реальному часі.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='ukraine-map',
                    figure=create_ukraine_map(),
                    style={'height': '200px'}
                ),
                html.Div(id='live-update-text'),
                dcc.Interval(
                    id='interval-component',
                    interval=60*1000,  # оновлення кожну хвилину
                    n_intervals=0
                )
            ]
        ),
        
        # Аналіз загроз
        html.Div(
            style={
                'backgroundColor': colors['threats'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Аналіз загроз', style={'textAlign': 'center', 'color': '#B54300'}),
                html.P('Оцінює потенційні ризики та загрози.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='threat-analysis',
                    figure=create_threat_analysis(),
                    style={'height': '200px'}
                )
            ]
        ),
        
        # Управління ресурсами
        html.Div(
            style={
                'backgroundColor': colors['resources'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Управління ресурсами', style={'textAlign': 'center', 'color': '#B50000'}),
                html.P('Контролює розподіл та використання ресурсів.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='resource-management',
                    figure=create_resource_management(),
                    style={'height': '200px'}
                ),
                html.Div(
                    id='resource-alerts',
                    style={'marginTop': '10px', 'color': 'red', 'textAlign': 'center'}
                )
            ]
        ),
        
        # Інструменти прогнозування
        html.Div(
            style={
                'backgroundColor': colors['forecasting'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Інструменти прогнозування', style={'textAlign': 'center', 'color': '#B5006A'}),
                html.P('Надає інструменти для прогнозного аналізу.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='forecast-chart',
                    figure=create_forecast_chart(),
                    style={'height': '200px'}
                )
            ]
        )
    ]),
    
    # Деталізована інформація
    html.Div(style={'marginTop': '30px'}, children=[
        html.H2('Деталізована інформація', style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Tabs([
            dcc.Tab(label='Карта України', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_ukraine_map())
                ])
            ]),
            dcc.Tab(label='Аналіз загроз війни', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_threat_analysis()),
                    html.Div([
                        html.H4('Контекст Російсько-Української війни'),
                        html.P('Дашборд відображає актуальні дані по різним типам загроз, пов\'язаних з воєнними діями. '
                               'Моніторинг включає як військові загрози (обстріли, атаки), так і супутні ризики для '
                               'цивільної інфраструктури та інформаційної безпеки.')
                    ])
                ])
            ]),
            dcc.Tab(label='Управління ресурсами', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_resource_management())
                ])
            ]),
            dcc.Tab(label='Прогнозування', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_forecast_chart())
                ])
            ])
        ])
    ])
])

# Оновлення інформації в реальному часі
@app.callback(
    Output('live-update-text', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_live_info(n):
    return html.Div([
        html.P(f"Останнє оновлення: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    ])

# Оновлення повідомлень про стан ресурсів
@app.callback(
    Output('resource-alerts', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_resource_alerts(n):
    critical_resources = resources_data[resources_data['Використання (%)'] > 80]['Ресурс'].tolist()
    if critical_resources:
        return html.P(f"Критичний рівень: {', '.join(critical_resources)}")
    return html.P("Усі ресурси в нормі")

# Запуск сервера - ВИПРАВЛЕННЯ: замінено run_server на run
if __name__ == '__main__':
    app.run(debug=True)  # Замість app.run_server(debug=True)

C:\Users\TZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

C:\Users\TZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [2]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import requests
from datetime import datetime, timedelta
from collections import Counter
import uuid

# Ініціалізація додатку Dash
app = dash.Dash(__name__)

# Стилі
colors = {
    'visualization': '#FFF9C4',  # світло-жовтий
    'threats': '#FFCCBC',        # світло-оранжевий
    'resources': '#FFCDD2',      # світло-червоний
    'forecasting': '#F8BBD0',    # світло-рожевий
    'background': '#F5F5F5',     # світло-сірий фон
    'text': '#333333'            # темно-сірий текст
}

# Завантаження GeoJSON
ukraine_geojson_url = "https://github.com/gadm/gadm36_levels_shp/raw/master/gadm36_UKR_shp/gadm36_UKR_1.shp"
response = requests.get(ukraine_geojson_url)
ukraine_geojson = response.json()

# Словник для відповідності назв регіонів
region_name_mapping = {
    'Київська': 'Kyiv',
    'Львівська': 'Lviv',
    'Харківська': 'Kharkiv',
    'Одеська': 'Odesa',
    'Дніпропетровська': 'Dnipropetrovsk',
    'Донецька': 'Donetsk',
    'Луганська': 'Luhansk',
    'Запорізька': 'Zaporizhia',
    'Херсонська': 'Kherson',
    'Миколаївська': 'Mykolaiv',
    'Вінницька': 'Vinnytsia',
    'Полтавська': 'Poltava',
    'Сумська': 'Sumy',
    'Чернігівська': 'Chernihiv',
    'Черкаська': 'Cherkasy',
    'Хмельницька': 'Khmelnytskyi',
    'Рівненська': 'Rivne',
    'Волинська': 'Volyn',
    'Івано-Франківська': 'Ivano-Frankivsk',
    'Тернопільська': 'Ternopil',
    'Закарпатська': 'Zakarpattia',
    'Чернівецька': 'Chernivtsi',
    'Житомирська': 'Zhytomyr',
    'Кіровоградська': 'Kirovohrad'
}

# Генерація тестових даних
def generate_test_data():
    regions = list(region_name_mapping.keys())
    security_levels = np.random.randint(10, 100, size=len(regions))
    high_risk_regions = ['Харківська', 'Донецька', 'Луганська', 'Запорізька', 'Херсонська', 'Сумська', 'Чернігівська']
    for region in high_risk_regions:
        if region in regions:
            idx = regions.index(region)
            security_levels[idx] = np.random.randint(10, 40)
    visualization_data = pd.DataFrame({'Область': regions, 'Рівень безпеки': security_levels})
    visualization_data['Region English'] = visualization_data['Область'].map(region_name_mapping)

    threat_types = [
        'Ракетні удари', 'Артилерійські обстріли', 'Безпілотники', 'Кібератаки',
        'Диверсійні групи', 'Інформаційні операції', 'Інфраструктурні атаки'
    ]
    risk_levels = np.random.randint(3, 10, size=len(threat_types))
    threat_categories = [
        'Військова загроза', 'Військова загроза', 'Військова загроза', 'Кібер загроза',
        'Фізична загроза', 'Інформаційна загроза', 'Критична інфраструктура'
    ]
    threats_data = pd.DataFrame({
        'Тип загрози': threat_types, 
        'Рівень ризику': risk_levels,
        'Категорія': threat_categories
    })

    resources = [
        'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
        'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
    ]
    usage = np.random.randint(50, 100, size=len(resources))
    resources_data = pd.DataFrame({'Ресурс': resources, 'Використання (%)': usage})

    dates = pd.date_range(start='2025-01-01', periods=30)
    forecasted = np.cumsum(np.random.normal(0, 2, size=len(dates))) + 50
    actual = forecasted[:20] + np.random.normal(0, 3, size=20)
    forecast_data = pd.DataFrame({
        'Дата': dates,
        'Прогноз': forecasted,
        'Фактично': np.append(actual, [None] * 10)
    })

    return visualization_data, threats_data, resources_data, forecast_data

viz_data, threats_data, resources_data, forecast_data = generate_test_data()

# Отримання даних ACLED
def fetch_acled_data():
    api_key = "your_api_key"  # Замініть на ваш ключ API
    email = "your_email"      # Замініть на вашу електронну пошту
    thirty_days_ago = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    # Для карти (усі події)
    url = f"https://api.acleddata.com/acled/read?key={api_key}&email={email}&country=Ukraine&event_date={thirty_days_ago}&event_date_where=>&fields=admin1"
    response = requests.get(url)
    data = response.json()
    events = data.get('data', [])
    region_counts = Counter([event['admin1'] for event in events])
    
    # Для аналізу загроз
    url_threats = f"https://api.acleddata.com/acled/read?key={api_key}&email={email}&country=Ukraine&event_type=Explosions/Remote violence&event_date={thirty_days_ago}&event_date_where=>&fields=sub_event_type"
    response_threats = requests.get(url_threats)
    data_threats = response_threats.json()
    threat_events = data_threats.get('data', [])
    sub_event_counts = Counter([event['sub_event_type'] for event in threat_events])
    
    return region_counts, sub_event_counts

region_counts, sub_event_counts = fetch_acled_data()

# Оновлення даних візуалізації
max_count = max(region_counts.values()) if region_counts else 1
for i, row in viz_data.iterrows():
    oblast = row['Область']
    english_name = region_name_mapping.get(oblast)
    count = region_counts.get(english_name, 0)
    security_level = 100 - (count / max_count * 90) if max_count > 0 else 100
    viz_data.at[i, 'Рівень безпеки'] = round(security_level, 1)

# Оновлення даних загроз
threat_mapping = {
    'Shelling/artillery/missile attack': ['Ракетні удари', 'Артилерійські обстріли'],
    'Air/drone strike': ['Безпілотники']
}
threat_risk = {threat: 0 for threat in threats_data['Тип загрози']}
for sub_event, count in sub_event_counts.items():
    if sub_event in threat_mapping:
        for threat in threat_mapping[sub_event]:
            threat_risk[threat] = count
max_threat_count = max(threat_risk.values()) if any(threat_risk.values()) else 0
for i, row in threats_data.iterrows():
    threat = row['Тип загрози']
    if threat in threat_risk:
        if max_threat_count > 0:
            scaled_risk = (threat_risk[threat] / max_threat_count) * 10
        else:
            scaled_risk = 0
        threats_data.at[i, 'Рівень ризику'] = round(scaled_risk, 1)

# Створення графіків
def create_ukraine_map():
    fig = px.choropleth(
        viz_data,
        geojson=ukraine_geojson,
        locations='Region English',
        featureidkey='properties.shapeName',
        color='Рівень безпеки',
        color_continuous_scale='RdYlGn',
        range_color=(10, 100),
        title='Оперативна ситуація в Україні'
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
    return fig

def create_threat_analysis():
    fig = px.bar(
        threats_data,
        x='Тип загрози',
        y='Рівень ризику',
        color='Категорія',
        color_discrete_map={
            'Військова загроза': '#FF5733',
            'Кібер загроза': '#33A1FF',
            'Фізична загроза': '#FF33FC',
            'Інформаційна загроза': '#33FF57',
            'Критична інфраструктура': '#FFD433'
        },
        title='Аналіз загроз (Російсько-Українська війна)'
    )
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis=dict(tickangle=45),
        yaxis=dict(title='Рівень загрози (1-10)')
    )
    return fig

def create_resource_management():
    fig = go.Figure()
    for i, row in resources_data.iterrows():
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=row['Використання (%)'],
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': row['Ресурс']},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkred" if row['Використання (%)'] > 80 else "orange" if row['Використання (%)'] > 60 else "green"},
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ))
    fig.update_layout(
        grid={'rows': 3, 'columns': 3, 'pattern': "independent"},
        title='Використання ресурсів',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig

def create_forecast_chart():
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Фактично'],
        mode='markers+lines',
        name='Фактичні дані',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Прогноз'],
        mode='lines',
        name='Прогноз',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title='Прогнозування потреб у ресурсах',
        xaxis_title='Дата',
        yaxis_title='Значення',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1(
        'Компоненти панелі управління',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'fontFamily': 'Arial',
            'marginBottom': '30px'
        }
    ),
    
    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-around'}, children=[
        html.Div(
            style={
                'backgroundColor': colors['visualization'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Візуалізація', style={'textAlign': 'center', 'color': '#B58500'}),
                html.P('Відображає оперативну ситуацію в реальному часі.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='ukraine-map',
                    figure=create_ukraine_map(),
                    style={'height': '200px'}
                ),
                html.Div(id='live-update-text'),
                dcc.Interval(
                    id='interval-component',
                    interval=60*1000,  # оновлення кожну хвилину
                    n_intervals=0
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['threats'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Аналіз загроз', style={'textAlign': 'center', 'color': '#B54300'}),
                html.P('Оцінює потенційні ризики та загрози.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='threat-analysis',
                    figure=create_threat_analysis(),
                    style={'height': '200px'}
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['resources'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Управління ресурсами', style={'textAlign': 'center', 'color': '#B50000'}),
                html.P('Контролює розподіл та використання ресурсів.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='resource-management',
                    figure=create_resource_management(),
                    style={'height': '200px'}
                ),
                html.Div(
                    id='resource-alerts',
                    style={'marginTop': '10px', 'color': 'red', 'textAlign': 'center'}
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['forecasting'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Інструменти прогнозування', style={'textAlign': 'center', 'color': '#B5006A'}),
                html.P('Надає інструменти для прогнозного аналізу.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='forecast-chart',
                    figure=create_forecast_chart(),
                    style={'height': '200px'}
                )
            ]
        )
    ]),
    
    html.Div(style={'marginTop': '30px'}, children=[
        html.H2('Деталізована інформація', style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Tabs([
            dcc.Tab(label='Карта України', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_ukraine_map())
                ])
            ]),
            dcc.Tab(label='Аналіз загроз війни', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_threat_analysis()),
                    html.Div([
                        html.H4('Контекст Російсько-Української війни'),
                        html.P('Дашборд відображає актуальні дані по різним типам загроз, пов\'язаних з воєнними діями. '
                               'Моніторинг включає як військові загрози (обстріли, атаки), так і супутні ризики для '
                               'цивільної інфраструктури та інформаційної безпеки.')
                    ])
                ])
            ]),
            dcc.Tab(label='Управління ресурсами', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_resource_management())
                ])
            ]),
            dcc.Tab(label='Прогнозування', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_forecast_chart())
                ])
            ])
        ])
    ])
])

# Оновлення інформації в реальному часі
@app.callback(
    Output('live-update-text', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_live_info(n):
    return html.Div([
        html.P(f"Останнє оновлення: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    ])

# Оновлення повідомлень про стан ресурсів
@app.callback(
    Output('resource-alerts', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_resource_alerts(n):
    critical_resources = resources_data[resources_data['Використання (%)'] > 80]['Ресурс'].tolist()
    if critical_resources:
        return html.P(f"Критичний рівень: {', '.join(critical_resources)}")
    return html.P("Усі ресурси в нормі")

# Запуск сервера
if __name__ == '__main__':
    app.run(debug=True)

JSONDecodeError: Expecting value: line 9 column 1 (char 8)